In [1]:
#Creating the functions
import pandas as pd

#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/lemmas.csv", sep = "{")
del lemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Dictionary_Dataframes/parses.csv", sep = "{")
del parsesDF["Unnamed: 0"]
#outputs list of matching IDs or an empty list
def get_id(word):
    #search the lemmaDF for the word
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if(row["id"] not in output):
            output.append(row["id"])
    return(output)
def get_id_parse(word, input):
    #search parseDF for instances and add to output if it isn't already in there
    output = input
    df = parsesDF.loc[parsesDF["bare_text"] == word]
    for index, row in df.iterrows():
        id = row["id"]
        if id not in output:
            output.append(id)
    return output

#outouts dictionary of ids to the valid words 
def get_words_help(ids):
    output = {}
    if(type(ids) == list):
        for id in ids:
            curOut = []
            df = parsesDF.loc[parsesDF["id"] == id]
            for index, row in df.iterrows():
                #can add a checker for POS or add info about POS
                curOut.append(row["bare_text"])
            output[id] = curOut
        return output
       #for each id in the list, grab all the words with matching ID from parses and put in "outputs"
       #outputs is dict of ID --> parses with ID 
    else:
        print("Something went wrong...")
        return {}
    #enter FPP
def get_id_full(word):
    ids = get_id(word)
    ids = get_id_parse(word, ids)
    return ids
def get_words(word):
    ids = get_id_full(word)
    return get_words_help(ids)
def check_enclitic(row, text):
    print("out of date")
def get_def(id):
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for index, row in df.iterrows():
        output = (row["definition"])
    return output
def get_lang(id):
    #returns ltn or grk
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        output = row["lang_id"]
    if(output == 3 ):
        output = "ltn"
    else:
        output = "grk"
    return output
def get_all_ids():
    #returns a list of all noun ids
    output = []
    #for every id in lemmas, add it to output
    for index, row in lemmaDF.iterrows():
        out = row["id"]
        #if an int, add it to the output
        if(type(out) == type(1)):
            output.append(out)
    return output

def make_dict_csv(ids, outputFile = "outputs/nounsDictionary.csv", allowNulls = True):
    #ids is the list of ids for all words - check for nouns in here 
    FPP = []
    POS = []
    DICTENTRY = []
    SOURCE = []
    for id in ids:
        pos = "" 
        fpp = ""
        dictEntry = ""
        source = "morpheus"
        text = {}
        df = parsesDF.loc[parsesDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        #if no words in parses, we can't make an entry for it -- current outlook
        if(df.empty):
            continue
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"] 
                #hold the plural genative just in case
                if(morph[2] == "s" and (morph[7] == "g")):
                    if(text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                        text[6] = row["bare_text"] 
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if(type(row["misc_features"]) == type(1.0) ):
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "m")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
                    
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "f")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "n")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4PP for verbs stored in text[6] - for weird words like futurus 
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(type(row["misc_features"]) == type(1.0)):
                    if(morph) == "v1spia---":
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]                    
                    if(morph) == "v--pna---":
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]
                    if(morph) == "v1sria---":
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
                    if(morph) == "v3spia---":
                        if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                            text[4] = row["bare_text"]
                    if(morph) == "v1spip---":
                        if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                            text[5] = row["bare_text"]
             #all other cases - simplest, just has 1 form 
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]  
        #set FPP to text[0]
        df2 = lemmaDF.loc[lemmaDF["id"] == id]
        if(pos == "noun"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            if(text[1] == "NULL"):
                text[1] = text[6]    
            dictEntry = text[0] + ", " + text[1] + ", " + text[2]
            fpp = text[0]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        elif(pos == "verb"):
            #change which type by checking if we have a normal FPP
            if(text[0]!= "NULL"):
                kind = "regular"
            elif(text[5] != "NULL"):
                kind = "deponent"
            else:
                kind = "impersonal"
                #nested switch block! Yay!
            #fix edge case - 4PP is weird
            if(text[3] == "NULL"):
                    text[3] = text[6]
            match(kind):
                case "regular":
                    fpp = text[0]
                    dictEntry = text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                case "deponent":
                    fpp = text[5]
                    dictEntry = text[5] + ", " + text[1] + ", " + text[3]
                case "impersonal":
                    fpp = text[4]
                    dictEntry = text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            if(fpp == "NULL"):
                for index, row in df2.iterrows():
                    fpp = row["bare_text"]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
           
        elif(pos == "adj"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + ", " + text[1] + ", " + text[2] 
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        else:
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + " indecl."
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)

    #end of function stuff
    out_map = {
        "FPP": FPP,
        "POS": POS,
        "Dictionary Entry": DICTENTRY,
        "Source": SOURCE
    }
    output = pd.DataFrame(out_map)
    output.to_csv(outputFile, sep = "\t")

            


        
        
#function from word -> dictionary entry
def get_dict(word, show_def = False):
    ids = get_id_full(word)
        #list of dictionary entries 
    output = {}
    for id in ids:
        lang = get_lang(id)
        if(lang == "grk"):
            continue
        pos = "" 
        dictEntry = ""
        rowText = ""
        morph = ""
        text = {}
        definition = get_def(id)
        df = parsesDF.loc[parsesDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns 
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6] 
                #make the nom, singular --> text[0]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "n")):
                    #have to hard-code around que enclitic 
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):

                        text[0] = row["bare_text"]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
            
            #adj case 
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if(type(row["misc_features"]) == type(1.0) ):
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "m")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
                    
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "f")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n") 
                    and (row["morph_code"][6] == "n")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"] 
            #backup 4th PP for verbs stored in text[6]
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"] 
            if((row["morph_code"][0] == "v")): 
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent 
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(type(row["misc_features"]) == type(1.0)):
                    if(morph) == "v1spia---":
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]                    
                    if(morph) == "v--pna---":
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]
                    if(morph) == "v1sria---":
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
                    if(morph) == "v3spia---":
                        if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                            text[4] = row["bare_text"]
                    if(morph) == "v1spip---":
                        if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                            text[5] = row["bare_text"]                    
            #all other cases
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
        #preventing errors <3 
        if(type(definition) != type("str") or len(definition) == 0):
            definition = "No Definiton"
            #case for when FPP isn't found in parses
        if(text[0] == "NULL"):
            df2 = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
        #make dictionary entry string
        match(pos):
            #make sure each of those exists before trying to output
            case "noun":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ". - " + (definition)
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "adj":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + " - " + definition
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] 
            case "verb":
                #change which type by checking if we have a normal FPP
                if(text[0]!= "NULL"):
                    kind = "regular"
                elif(text[5] != "NULL"):
                    kind = "deponent"
                else:
                    kind = "impersonal"
                    #nested switch block! Yay!
                #solve the edge case! 
                if(text[3] == "NULL"):
                    text[3] = text[6]
                match(kind):
                    case "regular":
                        if(show_def):
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                    case "deponent":
                        if(show_def):
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3]
                    case "impersonal":
                        if(show_def):
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            case other:
                dictEntry = pos + ": " + text[0]
                if(show_def):
                    dictEntry += " " + definition 
        output[id] = (dictEntry)
    return output

In [2]:
#new text block for getForm 
#load in the dataframes for functions
lemmaDF = pd.read_csv("Dictionary_Dataframes/lemmas.csv", sep = "{")
del lemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Dictionary_Dataframes/parses.csv", sep = "{")
del parsesDF["Unnamed: 0"]
def get_POS(id):
    #returns the part of speech of the word
    output = []
    df = parsesDF.loc[parsesDF["id"] == id]
    for(index, row) in df.iterrows():
        out = row["morph_code"][0]
        if out not in output:
            output.append(out)
    if "t" in output:
        output.remove("t")
        if "v" not in output:
            output.append("v")
    return output
def get_form(word = "NULL", id = "NULL", case = "NULL",
              number = "NULL", gender = "NULL", mood = "NULL", 
              person = "NULL", tense = "NULL", voice = "NULL",
                degree = "NULL", alt_dialects = False, wanted_pos = "NULL"):
    #check if word is null - use ID in that case
    if(word == "NULL" and id == "NULL"):
        print("Please enter a word or an ID")
        return "ERROR: Please enter a word or an ID"
    if(word!= "NULL" and id !="NULL"):
            print("Please enter a word or an ID, not both")
            return "ERROR: Please enter a word or an ID, not both"
    if(word != "NULL"):
        ids = get_id_full(word)
        if(len(ids) == 0):
            print("No matching words found")
            return "ERROR: No matching words"
    if(id != "NULL"):
        ids = [id]
    for id in ids:
        #if you find a valid output in one id, you're good. If not, we continue on 
        output = ""
        posTag = "---------"
        pos = get_POS(id)
        #if the word is a noun
        if("n" in pos):
            #noun 
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v" 
            
            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            
            #gender 
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
           
            #word assigned to output if it exists
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            # number
            if (number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            # gender 
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
        if("v" in pos):
            #verb
                #mood
            if(mood == "indicative"):
                mood = "i"
            if(mood == "subjunctive"):
                mood = "s"
            if(mood == "imperative"):
                mood = "m"
            if(mood == "infinitive"):
                mood = "n"
            if(mood == "participle"):
                mood = "p"
            if(mood == "gerund"):
                mood = "d"
            if(mood == "gerundive"):
                mood = "g"
            
                #person
            if(person == "first"):
                person = "1"
            if(person == "second"):
                person = "2"
            if(person == "third"):
                person = "3"
            
                #tense
            if(tense == "present"):
                tense = "p"
            if(tense == "imperfect"):
                tense = "i"
            if(tense == "perfect"):
                tense = "r"
            if(tense == "pluperfect"):
                tense = "l"
            if(tense == "future perfect"):
                tense = "t"
            if(tense == "future"):
                tense = "f"
            
                #voice
            if(voice == "active"):
                voice = "a"
            if(voice == "passive"):
                voice = "p"
            if(voice == "deponent"):
                voice = "d"
            
                #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
                # mood
            if(mood != "NULL"):
                posTag = posTag[:4] + mood + posTag[5:]
            # person
            if(person != "NULL"):
                posTag = posTag[:1] + person + posTag[2:]
            # tense
            if(tense != "NULL"):
                posTag = posTag[:3] + tense + posTag[4:]
            # voice
            if(voice != "NULL"):
                posTag = posTag[:5] + voice + posTag[6:]
            # number
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
        if("a" in pos):
                #adj case
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v" 
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            #gender 
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
            #degree
            if(degree == "positive"):
                degree = "p"
            if(degree == "comparative"):
                degree = "c"
            if(degree == "superlative"):
                degree = "s"
            if(degree != "NULL"):
                posTag = posTag[:8] + degree
        #set the pos to the wanted pos if it exists
        #change full words to their specific letters
        if(wanted_pos == "noun"):
            wanted_pos = "n"
        if(wanted_pos == "verb"):
            wanted_pos = "v"
        if(wanted_pos == "adjective"):
            wanted_pos = "a"
        if(wanted_pos == "adverb"):
            wanted_pos = "d"
        if(wanted_pos == "conjunction"):
            wanted_pos = "c"
        if(wanted_pos == "pronoun"):
            wanted_pos = "p"
        if(wanted_pos == "numeral"):
            wanted_pos = "m"
        if(wanted_pos == "interjection"):
            wanted_pos = "i"
        if(wanted_pos == "exclaimation"):
            wanted_pos = "e"
        if(wanted_pos == "punctuation"):
            wanted_pos = "u"
        #if the wanted pos is not null, change the posTag to match
        if(wanted_pos != "NULL"):
            posTag = wanted_pos + posTag[1:]
        #now actually find that word in parses 
        df = parsesDF.loc[parsesDF["id"] == id]
        for index, row in df.iterrows():
            fits = True
            if(wanted_pos == "v"):
                if(row["morph_code"][0] == "v" or row["morph_code"][0] == "t"):
                    fits = True
                else:
                    fits = False
                for i in range(1, 8):
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
            else:
                for i in range(8):
                    #check if alt dialects allowed
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results
            if fits:
                output = row["bare_text"]

        #before repeating for other ids, return the output if it exists
        if(output != ""):
            return output
        
    return "ERROR: Nothing found with those parameters"
        
        
    
        

In [3]:
def get_FPP(word = "NULL", id = "NULL", pos = "NULL"):
    #use char pos not full word
    if (word != "NULL"):
        ids = get_id_full(word)
        parts_of_speech = [] 
    if (id != "NULL"):
        ids = [id]


        #remove duplicates
    output = {}
    for id in ids: 
        output[id] = ""
        if (pos != "NULL"):
            parts_of_speech = [pos]
        else:
            parts_of_speech = get_POS(id)
        if("n" in parts_of_speech):
            temp = get_form(id = id, case = "nominative", number = "singular")
            if("ERROR" in temp):
                temp = get_form(id = id, case = "nominative", number = "plural")
            if("ERROR" not in temp):
                output[id] = temp
        if("v" in parts_of_speech or "t" in parts_of_speech):
            temp = get_form(id = id, mood = "indicative", person = "first", number = "singular", tense = "present", voice = "active")
            if("ERROR" in temp):
                #no singular 
                temp = get_form(id = id, mood = "indicative", person = "first", number = "plural", tense = "present", voice = "active")
            if("ERROR" in temp):
                #impersonal
                temp = get_form(id = id, mood = "indicative", person = "third", number = "singular", tense = "present", voice = "active")
            if("ERROR" in temp):
                #impersonal and no sing 
                temp = get_form(id = id, mood = "indicative", person = "third", number = "plural", tense = "present", voice = "active")
            if("ERROR" in temp):
                #deponent
                temp = get_form(id = id, mood = "indicative", person = "first", number = "singular", tense = "present")
            if("ERROR" not in temp):
                output[id] = temp
        if("a" in parts_of_speech):
            temp = get_form(id = id, case = "nominative", number = "singular", degree = "positive", gender = "masculine")
            if("ERROR" in temp):
                temp = get_form(id = id, case = "nominative", number = "singular", gender = "masculine")
            if("ERROR" not in temp):
                output[id] = temp

    return output

Run block 1 to initialize functions in kernel

get_words(str word) - returns a list of all words with that input in its list of parses

get_dict(str word, bool print_def = False) - returns all dictionary entries for a given word. 
    Gives definition if print_def is true

make_noun_dict(int[] ids, str outputfile = default path) - create a CSV that is a dictionary of all nouns in the database
make_dict_csv(string[] ids, string outputFile = "outputs/nounsDictionary.csv", bool include_null = true) - makes a dictionary. Options should be obv


getForm(word = "NULL", id = "NULL", case = "NULL", number = "NULL", gender = "NULL", mood = "NULL", person = "NULL", tense = "NULL", voice = "NULL", degree = "NULL", alt_dialect = False, wanted_pos = "NULL")
    for voice, if deponent verb, put deponent
    Only put what you're looking for, ie 
    get_form(word = "Mater", case = "ablative", number = "singular") will return Matre (should)
        working on allowing "ablative" or "b" to be used for ablative

get_FPP(id, word, pos)
    initializes all to know
    gets all of the FPP for each ID for a given word or just for a given ID 
    only uses a single POS if it's there 
    
    

In [13]:
#check if the code runs in another block
# print(get_id("mater"))
# print(get_words_help([46163]))
print(get_dict("matris", True))
print(get_form(word = "mater", case = "b", number = "s"))
print(get_form(word = "fluit", mood = "i", tense = "p", voice = "a", number = "s"))
print(get_form(word = "celer", gender = "f", number = "s", case = "a", degree = "p"))
    #fixed romanus giving romanum instead of orum - needed to disallow dialect differences
print(get_form(word = "Romanus", case = "g", number = "p", wanted_pos= "noun"))
print(get_form(word = "Romanus", case = "b", number = "s", wanted_pos= "a", gender = "f"))
print(get_FPP(word = "fluunt"))
print(get_form(word = "mater", case = "v", number = "s"))
print(get_form(word = "agricola", case = "a", number = "s"))

{46163: 'noun: mater, matris, f. - a mother', 46165: 'noun: mater, matris, f. - No Definiton'}
matre
fluo
celerem
romanorum
romana
{42821: 'fluo'}
mater
agricolam


In [21]:
# Function to compare original and lemmatized sentences
def compare_sentences(original_sentence, lemmatized_sentence):
    lemmatized_sentence_uppercase_first_letter = lemmatized_sentence[0].upper() + lemmatized_sentence[1:]
    print(f"Original:   {original_sentence}")
    print(f"Lemmatized: {lemmatized_sentence_uppercase_first_letter}")
    print()


# Testing Latin Sentence Generation


# Accusative Object
# [Subject: nom noun] + [Direct object: acc noun] + [Verb]
# Femina virum laudat. -> The woman praises the man.
nom_noun = get_form(word="femina", case="nominative", number="singular")
acc_noun = get_form(word="vir", case="accusative", number="singular")
verb = get_form(word="laudo", mood="indicative", person="third", number="singular", tense="present", voice="active")
sentence = f"{nom_noun} {acc_noun} {verb}."
print("Accusative Object")
print("English: The woman praises the man.")
compare_sentences("Femina virum laudat.", sentence)


# Two Accusative Objects
# [Subject: nom noun] + [Direct object 1: acc noun] + [Conjunction] + [Direct object 2: acc noun] + [Verb]
# Poeta pecuniam et famam habet. -> The poet has money and fame.
nom_noun = get_form(word="poeta", case="nominative", number="singular")
acc_noun1 = get_form(word="pecunia", case="accusative", number="singular")
conjunction = "et"  # "and"
acc_noun2 = get_form(word="fama", case="accusative", number="singular")
verb = get_form(word="habeo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun1} {conjunction} {acc_noun2} {verb}."
print("Two Accusative Objects")
print("English: The poet has money and fame.")
compare_sentences("Poeta pecuniam et famam habet.", sentence)

# Accusative Object + Dative Indirect Object
# [Subject: nom noun] + [Indirect object: dat noun] + [Direct object: acc noun] + [Verb]
# Agricola agro aquam adfert. -> The farmer brings water to the field.
nom_noun = get_form(word="agricola", case="nominative", number="singular")
dat_noun = get_form(word="ager", case="dative", number="singular")
acc_noun = get_form(word="aqua", case="accusative", number="singular")
verb = get_form(word="adfero", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {dat_noun} {acc_noun} {verb}."
print("Accusative Object + Dative Indirect Object")
print("English: The farmer brings water to the field.")
compare_sentences("Agricola agro aquam adfert.", sentence)

# Two Accusative Objects + Dative Indirect Object
# [Subject: nom noun] + [Indirect object: dat noun] + [Direct object 1: acc noun] + [Conjunction] + [Direct object 2: acc noun] + [Verb]
# Magister discipulo scientiam et sapientiam dat. -> The teacher gives knowledge and wisdom to the student.
nom_noun = get_form(word="magister", case="nominative", number="singular")
dat_noun = get_form(word="discipulus", case="dative", number="singular")
acc_noun1 = get_form(word="scientia", case="accusative", number="singular")
conjunction = "et"  # "and"
acc_noun2 = get_form(word="sapientia", case="accusative", number="singular")
verb = get_form(word="do", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {dat_noun} {acc_noun1} {conjunction} {acc_noun2} {verb}."
print("Two Accusative Objects + Dative Indirect Object")
print("English: The teacher gives knowledge and wisdom to the student.")
compare_sentences("Magister discipulo scientiam et sapientiam dat.", sentence)

# Statements with a Modifying Genitive of Possession
# [Subject: nom noun] + [Modifier: gen noun]
# Arbores silvae. -> The trees of the forest.
nom_noun = get_form(word="arbor", case="nominative", number="plural")
gen_noun = get_form(word="silva", case="genitive", number="singular")
sentence = f"{nom_noun} {gen_noun}."
print("Statements with a Modifying Genitive of Possession")
print("English: The trees of the forest.")
compare_sentences("Arbores silvae.", sentence)

# Statements with a Modifying Genitive of Possession (Direct Object)
# [Subject: nom noun] + [Direct object: acc noun] + [Modifier: gen noun] + [Verb]
# Rex cives urbis videt. -> The king sees the city’s citizens.
nom_noun = get_form(word="rex", case="nominative", number="singular")
acc_noun = get_form(word="civis", case="accusative", number="plural")
gen_noun = get_form(word="urbs", case="genitive", number="singular")
verb = get_form(word="video", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {gen_noun} {verb}."
print("Statements with a Modifying Genitive of Possession (Direct Object)")
print("English: The king sees the city’s citizens.")
compare_sentences("Rex cives urbis videt.", sentence)

# Statements with a Modifying Adverbial Ablative
# [Subject: nom noun] + [Modifier: abl noun] + [Verb]
# Miles gladio vincit. -> The soldier conquers by the sword.
nom_noun = get_form(word="miles", case="nominative", number="singular")
abl_noun = get_form(word="gladius", case="ablative", number="singular")
verb = get_form(word="vinco", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {abl_noun} {verb}."
print("Statements with a Modifying Adverbial Ablative")
print("English: The soldier conquers by the sword.")
compare_sentences("Miles gladio vincit.", sentence)

# Statements with a Modifying Adverbial Ablative (Direct Object)
# [Subject: nom noun] + [Direct object: acc noun] + [Modifier: abl noun] + [Verb]
# Poeta carmina stilo scribit. -> The poet writes poems with a stylus.
nom_noun = get_form(word="poeta", case="nominative", number="singular")
acc_noun = get_form(word="carmen", case="accusative", number="plural")
abl_noun = get_form(word="stilus", case="ablative", number="singular")
verb = get_form(word="scribo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {abl_noun} {verb}."
print("Statements with a Modifying Adverbial Ablative (Direct Object)")
print("English: The poet writes poems with a stylus.")
compare_sentences("Poeta carmina stilo scribit.", sentence)

# Statements with a Modifying Genitive and Ablative
# [Subject: nom noun] + [Modifier: gen noun] + [Modifier: abl noun] + [Verb]
# Filius agricolae rure venit. -> The farmer’s son is coming from the countryside.
nom_noun = get_form(word="filius", case="nominative", number="singular")
gen_noun = get_form(word="agricola", case="genitive", number="singular")
abl_noun = get_form(word="rus", case="ablative", number="singular")
verb = get_form(word="venio", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {gen_noun} {abl_noun} {verb}."
print("Statements with a Modifying Genitive and Ablative")
print("English: The farmer’s son is coming from the countryside.")
compare_sentences("Filius agricolae rure venit.", sentence)

# Statements with Adjectives
# [Subject: nom noun] + [Adjective: nom adj]
# Tyrannus avarus. -> A greedy tyrant.
nom_noun = get_form(word="tyrannus", case="nominative", number="singular")
nom_adj = get_form(word="avarus", case="nominative", number="singular", gender="masculine")
sentence = f"{nom_noun} {nom_adj}."
print("Statements with Adjectives")
print("English: A greedy tyrant.")
compare_sentences("Tyrannus avarus", sentence)

# Statements with Adjectives (Direct Object and Adjective)
# [Subject: nom noun] + [Adjective: nom adj] + [Direct object: acc noun] + [Adjective: acc adj] + [Verb]
# Canis magnus puerum parvum terret. -> The large dog scares the little boy.
nom_noun = get_form(word="canis", case="nominative", number="singular")
nom_adj = get_form(word="magnus", case="nominative", number="singular", gender="masculine")
acc_noun = get_form(word="puer", case="accusative", number="singular")
acc_adj = get_form(word="parvus", case="accusative", number="singular", gender="masculine")
verb = get_form(word="terreo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {nom_adj} {acc_noun} {acc_adj} {verb}."
print("Statements with Adjectives (Direct Object and Adjective)")
print("English: The large dog scares the little boy.")
compare_sentences("Canis magnus puerum parvum terret.", sentence)

# Predicate Adjectives
# [Subject: nom noun] + [Linking verb of being] + [Predicate: nom adj]
# Gladius est gravis. -> The sword is heavy.
nom_noun = get_form(word="gladius", case="nominative", number="singular")
linking_verb = get_form(word="sum", person="third", number="singular", tense="present", mood="indicative")
nom_adj = get_form(word="gravis", case="nominative", number="singular", gender="masculine")
sentence = f"{nom_noun} {linking_verb} {nom_adj}."
print("Predicate Adjectives")
print("English: The sword is heavy.")
compare_sentences("Gladius est gravis.", sentence)

# Substantive Adjectives
# [Subject: nom adj] + [Linking verb of being] + [Predicate: nom adj]
# Magni sunt sapientes. -> Great (men) are wise.
nom_adj = get_form(word="magnus", case="nominative", number="plural", gender="masculine")
linking_verb = get_form(word="sum", person="third", number="plural", tense="present", mood="indicative")
pred_adj = get_form(word="sapiens", case="nominative", number="plural", gender="masculine")
sentence = f"{nom_adj} {linking_verb} {pred_adj}."
print("Substantive Adjectives")
print("English: Great (men) are wise.")
compare_sentences("Magni sunt sapientes.", sentence)

# Substantive Adjectives with Direct Object
# [Subject: nom adj] + [Direct object: acc noun] + [Verb]
# Fortes civitates conservant. -> The brave (people) protect the citizens.
nom_adj = get_form(word="fortis", case="nominative", number="plural", gender="masculine")
acc_noun = get_form(word="civitas", case="accusative", number="plural")
verb = get_form(word="conservo", person="third", number="plural", tense="present", voice="active", mood="indicative")
sentence = f"{nom_adj} {acc_noun} {verb}."
print("Substantive Adjectives with Direct Object")
print("English: The brave (people) protect the citizens.")
compare_sentences("Fortes civitates conservant.", sentence)

# Demonstrative Adjectives
# [Subject demonstrative: nom adj] + [Subject: nom noun]
# Ille canis. -> That dog.
nom_adj = get_form(word="ille", case="nominative", number="singular", gender="masculine")
nom_noun = get_form(word="canis", case="nominative", number="singular")
sentence = f"{nom_adj} {nom_noun}."
print("Demonstrative Adjectives")
print("English: That dog.")
compare_sentences("Ille canis.", sentence)

# Demonstrative Adjectives with Direct Object
# [Subject demonstrative: nom adj] + [Subject: nom noun] + [Direct object demonstrative: acc adj] + [Direct object: acc noun] + [Verb]
# Hic vir illum canem laudet. -> This man praises that dog.
nom_adj1 = get_form(word="hic", case="nominative", number="singular", gender="masculine")
nom_noun = get_form(word="vir", case="nominative", number="singular")
acc_adj2 = get_form(word="ille", case="accusative", number="singular", gender="masculine")
acc_noun = get_form(word="canis", case="accusative", number="singular")
verb = get_form(word="laudo", person="third", number="singular", tense="present", voice="active", mood="subjunctive")
sentence = f"{nom_adj1} {nom_noun} {acc_adj2} {acc_noun} {verb}."
print("Demonstrative Adjectives with Direct Object")
print("English: This man praises that dog.")
compare_sentences("Hic vir illum canem laudet.", sentence)

Accusative Object
English: The woman praises the man.
Original:   Femina virum laudat.
Lemmatized: Femina uirum laudat.

Two Accusative Objects
English: The poet has money and fame.
Original:   Poeta pecuniam et famam habet.
Lemmatized: Poeta pecuniam et famam habet.

Accusative Object + Dative Indirect Object
English: The farmer brings water to the field.
Original:   Agricola agro aquam adfert.
Lemmatized: Agricola agro aquam ERROR: Nothing found with those parameters.

Two Accusative Objects + Dative Indirect Object
English: The teacher gives knowledge and wisdom to the student.
Original:   Magister discipulo scientiam et sapientiam dat.
Lemmatized: Magister discipulo scientiam et sapientiam dat.

Statements with a Modifying Genitive of Possession
English: The trees of the forest.
Original:   Arbores silvae.
Lemmatized: Arbores sylvae.

Statements with a Modifying Genitive of Possession (Direct Object)
English: The king sees the city’s citizens.
Original:   Rex cives urbis videt.
Lem

In [19]:
# Testing Latin Sentence Generation
# Adverbial Sentences

# General Adverbial Sentence
# [Subject: nom noun] + [Adverb] + [Verb]
# Pulla saepe studet. -> The girl often studies.
nom_noun = get_form(word="pulla", case="nominative", number="singular")
adverb = "saepe"  # "often"
verb = get_form(word="studeo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {adverb} {verb}."
print("General Adverbial Sentence")
print("English: The girl often studies.")
compare_sentences("Pulla saepe studet.", sentence)

# Adverbial Sentence with Direct Object
# [Subject: nom noun] + [Direct object: acc noun] + [Adverb] + [Verb]
# Miles moenia semper servat. -> A soldier always guards the city’s walls.
nom_noun = get_form(word="miles", case="nominative", number="singular")
acc_noun = get_form(word="moenia", case="accusative", number="plural")
adverb = "semper"  # "always"
verb = get_form(word="servo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {adverb} {verb}."
print("Adverbial Sentence with Direct Object")
print("English: A soldier always guards the city’s walls.")
compare_sentences("Miles moenia semper servat.", sentence)

# Adverbial Sentence with Direct Object (Specific Time)
# [Subject: nom noun] + [Direct object: acc noun] + [Adverb] + [Verb]
# Discipula libros heri reddidit. -> The student returned the books yesterday.
nom_noun = get_form(word="discipula", case="nominative", number="singular")
acc_noun = get_form(word="liber", case="accusative", number="plural")
adverb = "heri"  # "yesterday"
verb = get_form(word="reddo", person="third", number="singular", tense="perfect", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {adverb} {verb}."
print("Adverbial Sentence with Direct Object (Specific Time)")
print("English: The student returned the books yesterday.")
compare_sentences("Discipula libros heri reddidit.", sentence)

# Adverbial Sentence with Future Tense
# [Subject: nom noun] + [Direct object: acc noun] + [Adverb] + [Verb]
# Copiae hostem cras vinciet. -> The troops will overcome the enemy tomorrow.
nom_noun = get_form(word="copia", case="nominative", number="plural")
acc_noun = get_form(word="hostis", case="accusative", number="singular")
adverb = "cras"  # "tomorrow"
verb = get_form(word="vinco", person="third", number="singular", tense="future", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {adverb} {verb}."
print("Adverbial Sentence with Future Tense")
print("English: The troops will overcome the enemy tomorrow.")
compare_sentences("Copiae hostem cras vinciet.", sentence)

# Adverbial Sentence with Conjunction and Possessive
# [Subject: nom noun] + [Adj: nom adj] + [Conjunction] + [Possessive adj: abl] + [Object of conjunction: abl] + [Adverb] + [Verb]
# Tyrannus avarus cum suo pecunia numquam amittet. -> The greedy tyrant will never part with his money.
nom_noun = get_form(word="tyrannus", case="nominative", number="singular")
nom_adj = get_form(word="avarus", case="nominative", number="singular", gender="masculine")
conjunction = "cum"  # "with"
poss_adj = get_form(word="suus", case="ablative", number="singular", gender="feminine")
abl_noun = get_form(word="pecunia", case="ablative", number="singular")
adverb = "numquam"  # "never"
verb = get_form(word="amitto", person="third", number="singular", tense="future", voice="active", mood="indicative")
sentence = f"{nom_noun} {nom_adj} {conjunction} {poss_adj} {abl_noun} {adverb} {verb}."
print("dverbial Sentence with Conjunction and Possessive")
print("English: The greedy tyrant will never part with his money.")
compare_sentences("Tyrannus avarus cum suo pecunia numquam amittet.", sentence)

# Adverbial Question with Preposition
# [Adverb] + [Subject: nom noun] + [Preposition] + [Object of preposition: acc] + [Verb]
# Cur puer ad flumen it? -> Why does the boy go to the river?
adverb = "cur"  # "why"
nom_noun = get_form(word="puer", case="nominative", number="singular")
preposition = "ad"  # "to"
obj_prep = get_form(word="flumen", case="accusative", number="singular")
verb = get_form(word="eo", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{adverb} {nom_noun} {preposition} {obj_prep} {verb}?"
print("Adverbial Question with Preposition")
print("English: Why does the boy go to the river?")
compare_sentences("Cur puer ad flumen it?", sentence)

# Positive Degree Adverbial Sentence
# [Subject: nom noun] + [Positive adverb] + [Verb]
# Femina pulchre cantat. -> The woman sings beautifully.
nom_noun = get_form(word="femina", case="nominative", number="singular")
positive_adverb = "pulchre"  # "beautifully"
verb = get_form(word="canto", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {positive_adverb} {verb}."
print("Positive Degree Adverbial Sentence")
print("English: The woman sings beautifully.")
compare_sentences("Femina pulchre cantat.", sentence)

# Positive Degree Adverbial Sentence (Movement)
# [Subject: nom noun] + [Positive adverb] + [Verb]
# Canis celeriter currit. -> The dog runs quickly.
nom_noun = get_form(word="canis", case="nominative", number="singular")
positive_adverb = "celeriter"  # "quickly"
verb = get_form(word="curro", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {positive_adverb} {verb}."
print("Positive Degree Adverbial Sentence (Movement)")
print("English: The dog runs quickly.")
compare_sentences("Canis celeriter currit.", sentence)

# Comparative Degree Adverbial Sentence
# [Subject: nom noun] + [Comparative Adverb] + [Verb] + [Conjunction: quam] + [Object of comparison: nom noun]
# Miles fortius pugnat quam barbarus. -> The soldier is fighting more bravely than the barbarian.
nom_noun = get_form(word="miles", case="nominative", number="singular")
comparative_adverb = "fortius"  # "more bravely"
verb = get_form(word="pugno", person="third", number="singular", tense="present", voice="active", mood="indicative")
conjunction = "quam"  # "than"
comparison_noun = get_form(word="barbarus", case="nominative", number="singular")
sentence = f"{nom_noun} {comparative_adverb} {verb} {conjunction} {comparison_noun}."
print("Comparative Degree Adverbial Sentence")
print("English: The soldier is fighting more bravely than the barbarian.")
compare_sentences("Miles fortius pugnat quam barbarus.", sentence)

# Superlative Degree Adverbial Sentence
# [Subject: nom noun] + [Direct object: acc noun] + [Superlative adverb] + [Verb]
# Vir iaculum longissime iacet. -> The man throws the javelin the farthest.
nom_noun = get_form(word="vir", case="nominative", number="singular")
acc_noun = get_form(word="iaculum", case="accusative", number="singular")
superlative_adverb = "longissime"  # "the farthest"
verb = get_form(word="iacio", person="third", number="singular", tense="present", voice="active", mood="indicative")
sentence = f"{nom_noun} {acc_noun} {superlative_adverb} {verb}."
print("Superlative Degree Adverbial Sentence")
print("English: The man throws the javelin the farthest.")
compare_sentences("Vir iaculum longissime iacet.", sentence)

General Adverbial Sentence
English: The girl often studies.
Original:   Pulla saepe studet.
Lemmatized: Pulla saepe studet.

Adverbial Sentence with Direct Object
English: A soldier always guards the city’s walls.
Original:   Miles moenia semper servat.
Lemmatized: Miles moenia semper seruat.

Adverbial Sentence with Direct Object (Specific Time)
English: The student returned the books yesterday.
Original:   Discipula libros heri reddidit.
Lemmatized: Discipula libera heri reddidit.

Adverbial Sentence with Future Tense
English: The troops will overcome the enemy tomorrow.
Original:   Copiae hostem cras vinciet.
Lemmatized: Copiae hostem cras ERROR: Nothing found with those parameters.

dverbial Sentence with Conjunction and Possessive
English: The greedy tyrant will never part with his money.
Original:   Tyrannus avarus cum suo pecunia numquam amittet.
Lemmatized: Tyrannus auarus cum sua pecunia numquam amittet.

Adverbial Question with Preposition
English: Why does the boy go to the 